In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv


In [2]:
credits = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_credits.csv")
movies = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_movies.csv")
ratings = pd.read_csv("../input/the-movies-dataset/ratings_small.csv")

In [3]:
# column name changed

credits.columns = ['movieId','title','cast','crew']

movies.rename(columns={"id": "movieId"}, inplace=True)
#movies.columns

### what we need is that user id, movie id and rating

In [4]:

ratings = ratings.loc[:,["userId","movieId","rating"]]
ratings.head(10)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
5,1,1263,2.0
6,1,1287,2.0
7,1,1293,2.0
8,1,1339,3.5
9,1,1343,2.0


In [5]:
data = pd.merge(credits,movies, on="movieId")
df = data.merge(ratings, on="movieId")
df.columns

Index(['movieId', 'title_x', 'cast', 'crew', 'budget', 'genres', 'homepage',
       'keywords', 'original_language', 'original_title', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title_y', 'vote_average', 'vote_count', 'userId', 'rating'],
      dtype='object')

In [6]:
df.drop(['status', 'title_x', 'title_y'], axis=1, inplace=True)             # benzer attribute lari sildik

In [7]:
df.head(1)

,movieId,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,...,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,userId,rating
0,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",6.9,4500,39,4.0


### Lets see vote average

In [8]:
df.vote_average.mean()

6.802310053308838

In [9]:
# what we need is that movie id and title
df = df.loc[:,["movieId","original_title","userId","vote_average"]]
df.head(10)

,movieId,original_title,userId,vote_average
0,285,Pirates of the Caribbean: At World's End,39,6.9
1,559,Spider-Man 3,492,5.9
2,767,Harry Potter and the Half-Blood Prince,30,7.4
3,58,Pirates of the Caribbean: Dead Man's Chest,19,7.0
4,58,Pirates of the Caribbean: Dead Man's Chest,23,7.0
5,58,Pirates of the Caribbean: Dead Man's Chest,28,7.0
6,58,Pirates of the Caribbean: Dead Man's Chest,36,7.0
7,58,Pirates of the Caribbean: Dead Man's Chest,63,7.0
8,58,Pirates of the Caribbean: Dead Man's Chest,70,7.0
9,58,Pirates of the Caribbean: Dead Man's Chest,85,7.0


* As it can be seen data frame that is above, we have 4 features that are movie id, title user id and rating
* According to these data frame, we will make item based recommendation system

In [10]:
df.shape

(18571, 4)

* lets make a pivot table in order to make rows are users and columns are movies. And values are rating

In [11]:

pivot_table = df.pivot_table(index = ["userId"],columns = ["original_title"],values = "vote_average")
pivot_table.head(100)

original_title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,สุริโยไท,アキラ,七人の侍,三枪拍案惊奇,千と千尋の神隠し,卧虎藏龙,英雄,괴물,올드보이,친절한 금자씨
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,7.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,6.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.8,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.8,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* As it can be seen from table above, rows are users, columns are movies and values are ratings
* For example user 4 gives 6.8 rating to movie "20,000 Leagues Under the Sea" and user 8 gives 6.8 rating to same movie.
* Now lets make a scenario, we have movie web site and "2001: A Space Odyssey" movie are watched and rated by people. The question is that which movie do we recommend these people who watched "2001: A Space Odyssey" movie.
* In order to answer this question we will find similarities between "2001: A Space Odyssey" movie and other movies.



In [12]:
movie_watched = pivot_table["2001: A Space Odyssey"]


similarity_with_other_movies = pivot_table.corrwith(movie_watched)           
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
similarity_with_other_movies.head()

/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


original_title
Солярис           1.0
Secret Window     1.0
Open Water        1.0
Reindeer Games    1.0
Rize              1.0
dtype: float64


* It can be concluded that we need to recommend "Secret Window","Солярис","Open Water","Reindeer Games" and "Rize" movies to people who watched "2001: A Space Odyssey".
* On the other hand even if we do not consider, number of rating for each movie is also important.



# Conclusion

* If this tutorial is not enough you can check Recommendation System for Beginners prepared by 
    - https://www.kaggle.com/kanncaa1/recommendation-systems-tutorial
* After this tutorial, my aim is to prepare 'kernel' which is connected to Deep Learning with 'Sign Language MNSIT' data.
* If you have any suggestions, please could you write for me? I wil be happy for comment and critics!
* Thank you for your suggestion and votes ;)